In [1]:
import kaolin
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps
import diffvoronoi
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.loss_functions as lf
from pytorch3d.loss import chamfer_distance

#cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))

input_dims = 3
lr_sites = 0.005
#lr_model = 0.00001
destination = "./images/autograd/End2End_DCCVT_interpolSDF/"
model_trained_it = ""

mesh = ["gargoyle","/home/wylliam/dev/Kyushu_experiments/data/gargoyle"]
trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model{model_trained_it}.pth"
# 
# mesh = ["chair","/home/wylliam/dev/Kyushu_experiments/data/chair"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-05-02-17-56-25/chair/chair/trained_models/model{model_trained_it}.pth"

# mesh = ["bunny","/home/wylliam/dev/Kyushu_experiments/data/bunny"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-25-17-32-49/bunny/bunny/trained_models/model{model_trained_it}.pth"


Using device:  NVIDIA GeForce RTX 3090


In [2]:
from pytorch3d.ops import knn_points, knn_gather
import torch
from torch import nn

# class Voroloss_opt(nn.Module):
#     def __init__(self):
#         super(Voroloss_opt, self).__init__()
#         self.knn = 16

#     def __call__(self, points, spoints):
#         """points, self.points"""
#         # WARNING: fecthing for knn
#         with torch.no_grad():
#             indices = knn_points(points, spoints, K=self.knn).idx

#         points_knn = knn_gather(spoints, indices)
#         points_to_voronoi_center = points - points_knn[:, :, 0]

#         voronoi_edge = points_knn[:, :, 1:] - points_knn[:, :, 0].unsqueeze(2)
#         voronoi_edge_l = torch.sqrt(((voronoi_edge**2).sum(-1)))
#         vector_length = (points_to_voronoi_center.unsqueeze(2) * voronoi_edge).sum(
#             -1
#         ) / voronoi_edge_l
#         sq_dist = (vector_length - voronoi_edge_l / 2) ** 2
#         return sq_dist.min(-1)[0]

#voroloss = lf.Voroloss_opt().to(device)

In [3]:
num_centroids = 64**3
grid = 32
print("Creating new sites")
noise_scale = 0.05
domain_limit = 1
x = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
y = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
z = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
meshgrid = torch.meshgrid(x, y, z)
meshgrid = torch.stack(meshgrid, dim=3).view(-1, 3)


#add noise to meshgrid
meshgrid += torch.randn_like(meshgrid) * noise_scale


sites = meshgrid.to(device, dtype=torch.float32).requires_grad_(True)

print("Sites shape: ", sites.shape)
ps.init()


Creating new sites
Sites shape:  torch.Size([262144, 3])


/home/wylliam/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402412426/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 575.57.08


In [4]:
#LOAD MODEL WITH HOTSPOT
import sys
sys.path.append("3rdparty/HotSpot")
from dataset import shape_3d
import models.Net as Net

loss_type = "igr_w_heat"
loss_weights = [350, 0, 0, 1, 0, 0, 20]

train_set = shape_3d.ReconDataset(
    file_path = mesh[1]+".ply",
    n_points=grid*grid*150,#15000, #args.n_points,
    n_samples=10001, #args.n_iterations,
    grid_res=256, #args.grid_res,
    grid_range=1.1, #args.grid_range,
    sample_type="uniform_central_gaussian", #args.nonmnfld_sample_type,
    sampling_std=0.5, #args.nonmnfld_sample_std,
    n_random_samples=7500, #args.n_random_samples,
    resample=True,
    compute_sal_dist_gt=(
        True if "sal" in loss_type and loss_weights[5] > 0 else False
    ),
    scale_method="mean"#"mean" #args.pcd_scale_method,
)

model = Net.Network(
    latent_size=0,#args.latent_size,
    in_dim=3,
    decoder_hidden_dim=128,#args.decoder_hidden_dim,
    nl="sine",#args.nl,
    encoder_type="none",#args.encoder_type,
    decoder_n_hidden_layers=5,#args.decoder_n_hidden_layers,
    neuron_type="quadratic",#args.neuron_type,
    init_type="mfgi",#args.init_type,
    sphere_init_params=[1.6, 0.1],#args.sphere_init_params,
    n_repeat_period=30#args.n_repeat_period,
)
model.to(device)

######       
test_dataloader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=False, num_workers=0, pin_memory=False)   
test_data = next(iter(test_dataloader))
mnfld_points = test_data["mnfld_points"].to(device)
mnfld_points.requires_grad_()
print("mnfld_points shape: ", mnfld_points.shape)
if torch.cuda.is_available():
    map_location = torch.device("cuda")
else:
    map_location = torch.device("cpu")
model.load_state_dict(torch.load(trained_model_path, weights_only=True, map_location=map_location))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
mnfld_points shape:  torch.Size([1, 153600, 3])


/home/wylliam/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [5]:
# # #add mnfld points with random noise to sites 
# N = mnfld_points.squeeze(0).shape[0]
# num_samples = 64**3 - num_centroids
# idx = torch.randint(0, N, (num_samples,))
# sampled = mnfld_points.squeeze(0)[idx]
# perturbed = sampled + (torch.rand_like(sampled)-0.5)*0.05
# sites = torch.cat((sites, perturbed), dim=0)

# make sites a leaf tensor
sites = sites.detach().requires_grad_()
print(sites.dtype)
print(sites.shape)
print(f"Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

sdf0 = model(sites)
#sdf_grad0 = torch.autograd.grad(sdf0, sites, grad_outputs=torch.ones_like(sdf0), create_graph=True)[0]

sdf0 = sdf0.detach().squeeze(-1).requires_grad_()
#sdf_grad0 = sdf_grad0.detach().requires_grad_()
#offset = torch.zeros_like(sdf0).to(device)

print(sdf0.shape)
print(sdf0.is_leaf)

#print(sdf_grad0.shape)
#print(sdf_grad0.is_leaf)


torch.float32
torch.Size([262144, 3])
Allocated: 5.79072 MB, Reserved: 23.068672 MB
torch.Size([262144])
True


In [6]:
def sdf_site_gradients(
    sites: torch.Tensor,          # (N_sites, 3)  – xyz positions
    tets:  torch.LongTensor,      # (N_tets, 4)   – indices into `sites`
    sdf:   torch.Tensor,          # (N_sites,)    – φ-values at each site
    eps:   float = 1e-12
) -> torch.Tensor:
    """
    Vectorised re-write of `sdf_space_grad_cuda_kernel`.

    Returns
    -------
    grad : (N_sites, 3) torch.Tensor
        Volume-weighted ∇φ for every site, averaged over all tets incident
        to that site (the CUDA code accumulates   volume * elem   and then
        divides by   Σ volume ; we do the same).
    """
    # ---------------------------------------------------------------------
    # 1. Gather tetra vertex data
    # ---------------------------------------------------------------------
    v  = sites[tets]                 # (nT,4,3) vertices
    phi  = sdf[tets]                   # (nT,4)
    ctr = v.mean(dim=1, keepdim=True)  # (nT,1,3)
    phi_ctr = phi.mean(dim=1, keepdim=True) # (nT,1)

    dX = v - ctr                     # (nT,4,3)   x_i - x̄

    # ---------------------------------------------------------------------
    # 2. G = Σ dX_i dX_iᵀ   (nT,3,3)     and its inverse
    # ---------------------------------------------------------------------
    G   = torch.matmul(dX.transpose(1,2), dX)          # (nT,3,3)
    det = torch.det(G)                                 # (nT,)

    # Guard against degenerate tets
    mask = det.abs() > eps
    G_inv = torch.zeros_like(G)
    if mask.any():
        G_inv[mask] = torch.inverse(G[mask])

    # ---------------------------------------------------------------------
    # 3. Weights_curr  =  G⁻¹ dX_i        (nT,3,4)
    # ---------------------------------------------------------------------
    W = torch.matmul(G_inv, dX.transpose(1,2))         # (nT,3,4)

    # ---------------------------------------------------------------------
    # 4. elem = Σ (φ_i - φ̄) * W_i        (nT,3)
    # ---------------------------------------------------------------------
    grad_phi   = (phi - phi_ctr)                               # (nT,4)
    elem = (W * grad_phi.unsqueeze(1)).sum(dim=2)            # (nT,3)

    # ---------------------------------------------------------------------
    # 5. |T|  = volume of each tet       (nT,)
    #    |T| = | (a-d)·((b-d)×(c-d)) | / 6
    # ---------------------------------------------------------------------
    a, b, c, d = v[:,0], v[:,1], v[:,2], v[:,3]
    vol = torch.abs(
        torch.sum((a-d) * torch.cross(b-d, c-d, dim=1), dim=1)
    ) / 6.0                                           # (nT,)

    # ---------------------------------------------------------------------
    # 6. Scatter-add to per-site accumulators
    # ---------------------------------------------------------------------
    n_sites = sites.size(0)
    grad_accum   = torch.zeros(n_sites, 3, device=sites.device, dtype=sites.dtype)
    weight_accum = torch.zeros(n_sites, 1, device=sites.device, dtype=sites.dtype)

    # replicate elem*vol for the 4 vertices of every tet
    per_vtx_grad   = (elem * vol.unsqueeze(1)).unsqueeze(1).expand(-1,4,-1)  # (nT,4,3)
    per_vtx_weight = vol.unsqueeze(1)                                        # (nT,1)

    flat_ids   = tets.reshape(-1)                    # (nT*4,)
    flat_grad  = per_vtx_grad.reshape(-1, 3)         # (nT*4,3)
    flat_w     = per_vtx_weight.repeat(1,4).reshape(-1,1)  # (nT*4,1)

    grad_accum.scatter_add_(0, flat_ids.unsqueeze(-1).expand(-1,3), flat_grad)
    weight_accum.scatter_add_(0, flat_ids.unsqueeze(-1),                flat_w)

    # ---------------------------------------------------------------------
    # 7. Normalise: ∇φ_site = Σ |T|·elem  /  Σ |T|
    # ---------------------------------------------------------------------
    grad = grad_accum / weight_accum.clamp_min(eps)
    return grad


In [7]:
#sites_pred = model(sites).detach()#["nonmanifold_pnts_pred"]
#print(f"Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

ps_cloud = ps.register_point_cloud("initial_cvt_grid+pc_gt",sites.detach().cpu().numpy())
mnf_cloud = ps.register_point_cloud("mnfld_points_pred",mnfld_points.squeeze(0).detach().cpu().numpy())
ps_cloud.add_scalar_quantity("vis_grid_pred", sdf0.detach().cpu().numpy(), enabled=True, cmap="coolwarm", vminmax=(-0.00005, 0.00005))

# v_vect, f_vect = su.get_clipped_mesh_numba(sites, model, None, True)
# triangle_faces = [[f[0], f[i], f[i+1]] for f in f_vect for i in range(1, len(f)-1)]
# ps.register_surface_mesh("model clipped initial mesh", v_vect.detach().cpu().numpy(), triangle_faces)

v_vect, f_vect, sdf_verts, sdf_verts_grads = su.get_clipped_mesh_numba(sites, None, None, False, sdf0)
ps_mesh = ps.register_surface_mesh("sdf initial mesh", v_vect.detach().cpu().numpy(), f_vect)
ps_mesh.add_vector_quantity("sdf verts grads", sdf_verts_grads.detach().cpu().numpy(), enabled=True)

ps_vect = ps.register_point_cloud("sdf initial verts", v_vect.detach().cpu().numpy())
ps_vect.add_scalar_quantity("sdf verts values", sdf_verts.detach().cpu().numpy(), enabled=True, cmap="coolwarm")

v_vect, f_vect, sdf_verts, sdf_verts_grads = su.get_clipped_mesh_numba(sites, None, None, True, sdf0)
ps_mesh = ps.register_surface_mesh("sdf clipped initial mesh", v_vect.detach().cpu().numpy(), f_vect, back_face_policy="identical")
ps_mesh.add_vector_quantity("sdf verts grads", sdf_verts_grads.detach().cpu().numpy(), enabled=True)

ps_vect = ps.register_point_cloud("sdf clipped initial verts", v_vect.detach().cpu().numpy())
ps_vect.add_scalar_quantity("sdf verts values", sdf_verts.detach().cpu().numpy(), enabled=True, cmap="coolwarm")


ps.show()

In [ ]:
# SITES OPTIMISATION LOOP
cvt_loss_values = []
min_distance_loss_values = []
chamfer_distance_loss_values = []
eikonal_loss_values = []
domain_restriction_loss_values = []
sdf_loss_values = []
div_loss_values = []
loss_values = []


def train_DCCVT(sites, sites_sdf, offset=None, max_iter=100, stop_train_threshold=1e-6, upsampling=0, lambda_weights = [0.1,1.0,0.1,0.1,1.0,1.0,0.1]):
    optimizer = torch.optim.SGD([
    #{'params': [sites], 'lr': lr_sites},
    {'params': [sites_sdf], 'lr': lr_sites * 100},
    #{'params': [offset], 'lr': lr_sites},
    #{'params': model.parameters(), 'lr': lr_model}
])
    #scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80, 150, 200, 250], gamma=0.5)

    prev_loss = float("inf")
    best_loss = float("inf")
    upsampled = 0.0
    epoch = 0
    lambda_cvt = lambda_weights[0]
    lambda_chamfer = lambda_weights[4]
    best_sites = sites.clone()
    best_sites.best_loss = best_loss
    
    
    while epoch <= max_iter:
        print("sdf_values", sites_sdf[:5])
        
        optimizer.zero_grad()
        
        sites_np = sites.detach().cpu().numpy()
        d3dsimplices = diffvoronoi.get_delaunay_simplices(sites_np.reshape(input_dims*sites_np.shape[0]))
        d3dsimplices = np.array(d3dsimplices)

        # vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, None, None, d3dsimplices, sites_sdf)
        # vertices = su.compute_vertices_3d_vectorized(sites, vertices_to_compute)    
        # bisectors = su.compute_all_bisectors_vectorized(sites, bisectors_to_compute)
        # points = torch.cat((vertices, bisectors), 0)
    
        #cvt_loss = lf.compute_cvt_loss_vectorized_delaunay(sites, None, d3dsimplices)
        #print("CVT loss: ", cvt_loss, "weighted: ", lambda_cvt*cvt_loss)

        # from pytorch3d.loss import chamfer_distance
        # chamfer_loss_points, _ = chamfer_distance(mnfld_points.detach(), points.unsqueeze(0))
        # print(f"Points Chamfer loss PYTORCH3D {chamfer_loss_points} weighted: {lambda_chamfer*chamfer_loss_points} : Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

        #sites_sdf_grads = sdf_site_gradients(sites, torch.tensor(d3dsimplices, device=device), sites_sdf)
        #sites_positions = sites.detach().clone()
        
        v_vect, f_vect, sdf_verts, sdf_verts_grads = su.get_clipped_mesh_numba(sites, None, d3dsimplices, True, sites_sdf, offset)
        
        ps_mesh = ps.register_surface_mesh(f"{epoch} sdf clipped pmesh", v_vect.detach().cpu().numpy(), f_vect, back_face_policy="identical")
        ps_mesh.add_scalar_quantity(f"{epoch} sdf clipped pmesh", sdf_verts.detach().cpu().numpy(), enabled=True, cmap="coolwarm")
        ps_mesh.add_vector_quantity(f"{epoch} sdf verts grads", sdf_verts_grads.detach().cpu().numpy(), enabled=True)

        triangle_faces = [[f[0], f[i], f[i+1]] for f in f_vect for i in range(1, len(f)-1)]
        triangle_faces = torch.tensor(triangle_faces, device=device)
        hs_p = su.sample_mesh_points_heitz(v_vect, triangle_faces, num_samples=32*32*150)
        chamfer_loss_mesh, _ = chamfer_distance(mnfld_points.detach(), hs_p.unsqueeze(0))
        
        # chamfer_loss_mesh, _ = chamfer_distance(mnfld_points.detach(), v_vect.unsqueeze(0))
        
        print(f"Mesh Chamfer loss PYTORCH3D {chamfer_loss_mesh} weighted: {lambda_chamfer*chamfer_loss_mesh} : Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

        
        sites_loss = (
            #lambda_cvt * cvt_loss +
            lambda_chamfer * chamfer_loss_mesh 
            #lambda_chamfer * chamfer_loss_points
            #lambda_chamfer * voroloss_loss
        )

        # voroloss_loss = voroloss(mnfld_points.squeeze(0), sites).mean()
        # sites_loss = (
        #     lambda_chamfer * voroloss_loss
        # )

        loss = sites_loss #+ sdf_loss
        loss_values.append(loss.item())
        print(f"Epoch {epoch}: loss = {loss.item()}")
        
        print(f"before loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        loss.backward()
        print(f"After loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        print("-----------------")
        
        
        optimizer.step()

        #sites_sdf += (sites_sdf_grads*(sites-sites_positions)).sum(dim=1)
                
        
        #scheduler.step()

        
        # if epoch>100 and (epoch // 100) == upsampled+1 and loss.item() < 0.5 and upsampled < upsampling:
       
        # if epoch/max_iter > (upsampled+1)/(upsampling+1) and upsampled < upsampling:
        #     print("sites length BEFORE UPSAMPLING: ",len(sites))
        #     sites = su.upsampling_vectorized(sites, tri=None, vor=None, simplices=d3dsimplices, model=model)
        #     sites = sites.detach().requires_grad_(True)
        #     optimizer = torch.optim.Adam([{'params': [sites], 'lr': lr_sites}, 
        #                                   #{'params': model.parameters(), 'lr': lr_model}
        #                                   ])
        #     upsampled += 1.0
        #     print("sites length AFTER: ",len(sites))
        
          
        if epoch % (max_iter/10) == 0 or epoch == max_iter:
            #print(f"Epoch {epoch}: loss = {loss.item()}")
            #print(f"Best Epoch {best_epoch}: Best loss = {best_loss}")
            #save model and sites
            #ps.register_surface_mesh(f"{epoch} triangle clipped mesh", v_vect.detach().cpu().numpy(), triangle_faces.detach().cpu().numpy())
            
            #ps.register_point_cloud('sampled points end', hs_p.detach().cpu().numpy())
            ps.register_point_cloud('sampled points end', v_vect.detach().cpu().numpy())
            
                
            site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}.pth'
            #model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lambda_chamfer}.pth'
            sdf_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sdf_{num_centroids}_chamfer{lambda_chamfer}.pth'
            torch.save(sites_sdf, sdf_file_path)
            torch.save(sites, site_file_path)
            
        epoch += 1           

    return sites, sites_sdf

In [9]:
#lambda_weights = [252,0,0,0,10.211111,0,100,0]
#lambda_weights = [500,0,0,0,1000,0,100,0]
lambda_weights = [100,0,0,0,1000,0,100,0]


lambda_cvt = lambda_weights[0]
lambda_sdf = lambda_weights[1]
lambda_min_distance = lambda_weights[2]
lambda_laplace = lambda_weights[3]
lambda_chamfer = lambda_weights[4]
lambda_eikonal = lambda_weights[5]
lambda_domain_restriction = lambda_weights[6]
lambda_true_points = lambda_weights[7]

max_iter = 10

In [10]:
site_file_path = f'{destination}{max_iter}_cvt_{lambda_cvt}_chamfer_{lambda_chamfer}_eikonal_{lambda_eikonal}.npy'
#check if optimized sites file exists
if not os.path.exists(site_file_path):
    #import sites
    print("Importing sites")
    sites = np.load(site_file_path)
    sites = torch.from_numpy(sites).to(device).requires_grad_(True)
else:
    # import cProfile, pstats
    # import time
    # profiler = cProfile.Profile()
    # profiler.enable()

#     with torch.profiler.profile(activities=[
#             torch.profiler.ProfilerActivity.CPU,
#             torch.profiler.ProfilerActivity.CUDA,
#         ],
#         record_shapes=False,
#         with_stack=True  # Captures function calls
#     ) as prof:
#         sites = train_DCCVT(sites, model, max_iter=max_iter, upsampling=1, lambda_weights=lambda_weights)
#         torch.cuda.synchronize()
# # 
#     print(prof.key_averages().table(sort_by="self_cuda_time_total"))
#     prof.export_chrome_trace("trace.json")
    
    # 
    sites, optimized_sites_sdf = train_DCCVT(sites, sdf0, offset=None, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights)
    # print("Offset:", torch.min(offset), torch.max(offset))
    # print("Offset mean:", torch.mean(offset))
    
    # profiler.disable()
    # stats = pstats.Stats(profiler).sort_stats('cumtime')
    # stats.print_stats()
    # stats.dump_stats(f'{destination}{mesh[0]}{max_iter}_3d_profile_{num_centroids}_chamfer{lambda_chamfer}.prof')
    
    
    sites_np = sites.detach().cpu().numpy()
    np.save(site_file_path, sites_np)
    
print("Sites length: ", len(sites))
print("min sites: ", torch.min(sites))
print("max sites: ", torch.max(sites))

sdf_values tensor([1.2222, 1.2698, 1.2132, 1.1966, 1.2119], device='cuda:0',
       grad_fn=<SliceBackward0>)
Mesh Chamfer loss PYTORCH3D 8.456445357296616e-05 weighted: 0.0845644548535347 : Allocated: 3159.384064 MB, Reserved: 3856.662528 MB
Epoch 0: loss = 0.0845644548535347
before loss.backward(): Allocated: 3159.384576 MB, Reserved: 3856.662528 MB
After loss.backward(): Allocated: 1601.033728 MB, Reserved: 3856.662528 MB
-----------------
sdf_values tensor([1.2222, 1.2698, 1.2132, 1.1966, 1.2119], device='cuda:0',
       grad_fn=<SliceBackward0>)
Mesh Chamfer loss PYTORCH3D 0.01597399264574051 weighted: 15.973992347717285 : Allocated: 3175.978496 MB, Reserved: 3858.75968 MB
Epoch 1: loss = 15.973992347717285
before loss.backward(): Allocated: 3175.978496 MB, Reserved: 3858.75968 MB
After loss.backward(): Allocated: 1604.954112 MB, Reserved: 3858.75968 MB
-----------------
sdf_values tensor([1.2222, 1.2698, 1.2132, 1.1966, 1.2119], device='cuda:0',
       grad_fn=<SliceBackward0>)
M

In [11]:
epoch = 10

#model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lambda_chamfer}.pth'
site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}.pth'
sdf_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sdf_{num_centroids}_chamfer{lambda_chamfer}.pth'


 
sites = torch.load(site_file_path)
sdf_v = torch.load(sdf_file_path)

sites_np = sites.detach().cpu().numpy()
print("sdf", sdf_v.shape)
print("sites", site_file_path)

ps_cloud_f = ps.register_point_cloud(f"{epoch} epoch_cvt_grid", sites_np)
ps_cloud_f.add_scalar_quantity("vis_grid_pred", sdf_v.detach().cpu().numpy(), enabled=True, cmap="coolwarm", vminmax=(-0.15, 0.15))

print("sites_np shape: ", sites_np.shape)

#print sites if Nan
if np.isnan(sites_np).any():
    print("sites_np contains NaN values")
    print("sites_np NaN values: ", np.isnan(sites_np).sum())
#remove nan values from sites tensor
sites_np = sites_np[~np.isnan(sites_np).any(axis=1)]
sites = torch.from_numpy(sites_np).to(device).requires_grad_(True)

sdf torch.Size([262144])
sites ./images/autograd/End2End_DCCVT_interpolSDF/gargoyle10_10_3d_sites_262144_chamfer1000.pth
sites_np shape:  (262144, 3)


In [12]:
#v_vect, f_vect = su.get_clipped_mesh_numba(sites, model, None, True)
#ps.register_surface_mesh("model final clipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)

# v_vect, f_vect = su.get_clipped_mesh_numba(sites, model, None, False)
# ps.register_surface_mesh("model final polygon mesh", v_vect.detach().cpu().numpy(), f_vect)

######################################################
# v_vect, f_vect = su.get_clipped_mesh_numba(sites, None, None, True, sdf_v)
# ps.register_surface_mesh("sdf final clipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect) 

ps.show()